In [12]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.models import load_model
from IPython.display import Image

In [ ]:
IMAGE_SIZE = [224, 224]
train_path = './drive/MyDrive/Facial Recognition/Datasets/train'
valid_path = './drive/MyDrive/Facial Recognition/Datasets/test'


In [ ]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# useful for getting number of classes
folders = glob('./drive/MyDrive/Facial Recognition/Datasets/train/*')
folders

['./drive/MyDrive/Facial Recognition/Datasets/train/Newsun',
 './drive/MyDrive/Facial Recognition/Datasets/train/Niraj']

In [ ]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./drive/MyDrive/Facial Recognition/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./drive/MyDrive/Facial Recognition/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 200 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
7/7 [==============================] - 185s 25s/step - loss: 1.0343 - accuracy: 0.5347 - val_loss: 0.7062 - val_accuracy: 0.5667
Epoch 2/5
7/7 [==============================] - 130s 19s/step - loss: 0.2212 - accuracy: 0.9180 - val_loss: 0.1226 - val_accuracy: 0.9833
Epoch 3/5
7/7 [==============================] - 130s 19s/step - loss: 0.0695 - accuracy: 0.9823 - val_loss: 0.0832 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 130s 19s/step - loss: 0.0401 - accuracy: 0.9757 - val_loss: 0.5918 - val_accuracy: 0.7167
Epoch 5/5
7/7 [==============================] - 130s 19s/step - loss: 0.0286 - accuracy: 1.0000 - val_loss: 0.0807 - val_accuracy: 1.0000


In [ ]:
model.save('./drive/MyDrive/Facial Recognition/VGG16_model.h5')